In [1]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#import the neccessary module
from helper import Model
from helper import Auxiliary

# Modelling
from keras.models import Sequential
from keras import layers
from keras.backend import clear_session

set the data


In [2]:
#df_train, df_test = Auxiliary.train_test_split(Auxiliary.preprocess_data(Model.get_csv('BPI_Challenge_2012.csv')))
#read the data set
df_train = Auxiliary.preprocess_data(Model.get_csv('BPI_Challenge_2012_train.csv'))
df_test = Auxiliary.preprocess_data(Model.get_csv('BPI_Challenge_2012_test.csv'))

Start preprocessing data
Finished preprocessing data
Start preprocessing data
Finished preprocessing data


In [3]:

# encode the concept:name as integer values
df_encoded_train = df_train.copy()

lengthOfDf = len(df_encoded_train.index)

#add a new column with the position of each case
df_encoded_train.at[0, 'position'] = 1
j = 2
for i in range(1, lengthOfDf):
    if df_encoded_train.at[i, 'case:concept:name'] == df_encoded_train.at[i - 1, 'case:concept:name']:
        df_encoded_train.at[i, 'position'] = j
        j = j + 1
    else :
        df_encoded_train.at[i, 'position'] = 1
        j = 2


#the mode of the activities at a certain position
df_mode = df_encoded_train.groupby('position')['concept:name'].agg(pd.Series.mode)

#add a new column with the predicted time of each case and the ground truth
for i in range(lengthOfDf - 1):
    if df_encoded_train.at[i, 'position'] + 1 <= len(df_mode.index):
        if (df_encoded_train.at[i + 1, 'position']) == 1:
            df_encoded_train.at[i, 'Next time'] = 0
        else:
            df_encoded_train.at[i, 'Next time'] = Model.get_time_difference_as_number(df_encoded_train.at[i, 'time:timestamp'], df_encoded_train.at[i + 1, 'time:timestamp'])


In [4]:

# encode the concept:name as integer values
df_encoded_test = df_test.copy()
lengthOfDf = len(df_encoded_test.index)

#add a new column with the position of each case
df_encoded_test.at[0, 'position'] = 1
j = 2
for i in range(1, lengthOfDf):
    if df_encoded_test.at[i, 'case:concept:name'] == df_encoded_test.at[i - 1, 'case:concept:name']:
        df_encoded_test.at[i, 'position'] = j
        j = j + 1
    else :
        df_encoded_test.at[i, 'position'] = 1
        j = 2


#the mode of the activities at a certain position
df_mode = df_encoded_test.groupby('position')['concept:name'].agg(pd.Series.mode)

#add a new column with the predicted time of each case and the ground truth
for i in range(lengthOfDf - 1):
    if df_encoded_test.at[i, 'position'] + 1 <= len(df_mode.index):
        if (df_encoded_test.at[i + 1, 'position']) == 1:
            df_encoded_test.at[i, 'Next time'] = 0
        else:
            df_encoded_test.at[i, 'Next time'] = Model.get_time_difference_as_number(df_encoded_test.at[i, 'time:timestamp'], df_encoded_test.at[i + 1, 'time:timestamp'])

In [5]:

data_to_drop = ['org:resource', 'lifecycle:transition', 'case:AMOUNT_REQ', 'Next Event', 'case:REG_DATE', 'concept:name', 'case:concept:name', 'time:timestamp', 'position', 'minute',
       'second','year', 'month', 'hour']

#split the data into training and test sets and drop some data
x_train = df_encoded_train.drop(data_to_drop, axis=1)
x_train = x_train.drop(['W_Wijzigen contractgegevens'], axis = 1)
x_test = df_encoded_test.drop(data_to_drop, axis=1)
x_train = x_train.drop(['Next time'], axis=1)
x_test = x_test.drop(['Next time'], axis=1)

y_train = df_encoded_train['Next time']
y_test = df_encoded_test['Next time']

#fill all empty values with nan
x_train = x_train.replace(r'^\s*$', np.nan, regex=True)
x_test = x_test.replace(r'^\s*$', np.nan, regex=True)
y_train = y_train.replace(r'^\s*$', np.nan, regex=True)
y_test = y_test.replace(r'^\s*$', np.nan, regex=True)


#Fill all nan values with 0
x_train = x_train.replace(np.nan,0)
x_test = x_test.replace(np.nan,0)
y_train = y_train.replace(np.nan,0)
y_test = y_test.replace(np.nan,0)





x_test = x_test[['A_SUBMITTED', 'A_PARTLYSUBMITTED', 'A_PREACCEPTED',
       'W_Completeren aanvraag', 'A_ACCEPTED', 'O_SELECTED', 'A_FINALIZED',
       'O_CREATED', 'O_SENT', 'W_Nabellen offertes', 'O_SENT_BACK',
       'W_Valideren aanvraag', 'A_REGISTERED', 'A_APPROVED', 'O_ACCEPTED',
       'A_ACTIVATED', 'O_CANCELLED', 'A_DECLINED', 'A_CANCELLED',
       'W_Afhandelen leads', 'O_DECLINED', 'W_Nabellen incomplete dossiers',
       'W_Beoordelen fraude','day']]

In [6]:
print(x_test.head(10))
print(y_test.head(10))

   A_SUBMITTED  A_PARTLYSUBMITTED  A_PREACCEPTED  W_Completeren aanvraag  \
0            1                  0              0                       0   
1            0                  1              0                       0   
2            0                  0              0                       0   
3            0                  0              0                       0   
4            0                  0              0                       0   
5            0                  0              0                       0   
6            1                  0              0                       0   
7            0                  1              0                       0   
8            0                  0              0                       0   
9            0                  0              0                       0   

   A_ACCEPTED  O_SELECTED  A_FINALIZED  O_CREATED  O_SENT  \
0           0           0            0          0       0   
1           0           0            0   

In [22]:
#now everything is in array format
train_x =np.array(x_train)
train_y = np.array(y_train)
test_x = np.array(x_test)
test_y = np.array(y_test)

train_x = np.asarray(train_x).astype('float32')
train_y = np.asarray(train_y).astype('float32')
test_x = np.asarray(test_x).astype('float32')
test_y = np.asarray(test_y).astype('float32')


In [9]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(38862, 24)
(38862,)
(17201, 24)
(17201,)


In [10]:

input_dim = x_train.shape[1] # Number of features

In [11]:
clear_session()

In [14]:
from keras.layers import Flatten, Dense
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy

In [15]:
def model_builder(hp):
  model = Sequential()
  #model.add(Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(Dense(units=hp_units, activation='relu'))
  model.add(Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                loss=SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [16]:
import keras_tuner as kt

In [19]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=100,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [20]:
import tensorflow as tf
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [28]:
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

<tf.Tensor: shape=(17201,), dtype=float32, numpy=
array([1.0900000e-01, 9.7060003e+00, 6.3927280e+03, ..., 4.8579416e+05,
       1.6247000e+01, 0.0000000e+00], dtype=float32)>

In [29]:
tuner.search(x_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
384               |?                 |units
0.0001            |?                 |learning_rate
2                 |?                 |tuner/epochs
0                 |?                 |tuner/initial_epoch
4                 |?                 |tuner/bracket
0                 |?                 |tuner/round

Epoch 1/2


Traceback (most recent call last):
  File "c:\Users\Cristian\AppData\Local\Programs\Python\Python310\lib\site-packages\keras_tuner\engine\base_tuner.py", line 266, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\Cristian\AppData\Local\Programs\Python\Python310\lib\site-packages\keras_tuner\engine\base_tuner.py", line 231, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\Cristian\AppData\Local\Programs\Python\Python310\lib\site-packages\keras_tuner\tuners\hyperband.py", line 419, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\Cristian\AppData\Local\Programs\Python\Python310\lib\site-packages\keras_tuner\engine\tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "c:\Users\Cristian\AppData\Local\Programs\Python\Python310\lib\site-packages\keras_tuner\engine\tuner.py", lin

RuntimeError: Number of consecutive failures excceeded the limit of 3.
Traceback (most recent call last):
  File "c:\Users\Cristian\AppData\Local\Programs\Python\Python310\lib\site-packages\keras_tuner\engine\base_tuner.py", line 266, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\Cristian\AppData\Local\Programs\Python\Python310\lib\site-packages\keras_tuner\engine\base_tuner.py", line 231, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\Cristian\AppData\Local\Programs\Python\Python310\lib\site-packages\keras_tuner\tuners\hyperband.py", line 419, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\Cristian\AppData\Local\Programs\Python\Python310\lib\site-packages\keras_tuner\engine\tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "c:\Users\Cristian\AppData\Local\Progra